In [1]:


import pandas as pd
import datetime

from IPython.html import widgets 
from IPython.display import display, clear_output

import plotly.plotly as py
from plotly.graph_objs import *
import plotly
from plotly.widgets import GraphWidget



/usr/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/usr/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



<IPython.core.display.Javascript object>

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/widgets/master/ipython-examples/311_150k.csv', parse_dates=True, index_col=1)
df = df
df.head()

/usr/lib64/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning:

Columns (8,39,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.



,Unique Key,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
Created Date,,,,,,,,,,,,,,,,,,,,,
2014-11-16 23:46:00,29300358,11/16/2014 11:46:00 PM,DSNY,BCC - Queens East,Derelict Vehicles,14 Derelict Vehicles,Street,11432,80-25 PARSONS BOULEVARD,PARSONS BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.719411,-73.808882,"(40.719410639341916, -73.80888158860446)"
2014-11-16 02:24:35,29299837,11/16/2014 02:24:35 AM,DOB,Department of Buildings,Building/Use,Illegal Conversion Of Residential Building/Space,NaN,10465,938 HUNTINGTON AVENUE,HUNTINGTON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.827862,-73.830641,"(40.827862046105416, -73.83064067165407)"
2014-11-16 02:17:12,29297857,11/16/2014 02:50:48 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11201,229 DUFFIELD STREET,DUFFIELD STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.691248,-73.984375,"(40.69124772858873, -73.98437529459297)"
2014-11-16 02:15:13,29294647,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10040,128 NAGLE AVENUE,NAGLE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.861248,-73.926308,"(40.861247930170535, -73.92630783362215)"
2014-11-16 02:14:01,29300211,NaN,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,10306,625 LINCOLN AVENUE,LINCOLN AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.570565,-74.092229,"(40.57056460126485, -74.09222907551542)"


In [3]:
grouped = df.resample('24H', how='count')

In [5]:


slider = widgets.IntSlider()
slider.min = 1
slider.value=24
slider.description = 'Window length (hours)'



In [7]:
column_headers_dropdown = widgets.Dropdown(
    options  = {column: column for column in df.columns},
    selected_label = 'Complaint Type',
    description = 'Select which column to graph (e.g. Tree)'
)


search_complaints_text_input = widgets.Text()
search_complaints_text_input.description = 'Search complaint types'

In [8]:
graph_time_series_agg = GraphWidget('https://plot.ly/~chris/4103')
graph_time_series_agg.ranges = {'x': [1413648000000, 1416157200000], 'y': [0, 7744.210526315789]}
graph_complaints = GraphWidget('https://plot.ly/~chris/4103')


In [9]:
def replot(on_zoom=False):
    search_text = search_complaints_text_input.value
    slider_val = slider.value
    column_name = column_headers_dropdown.selected_label

    # Time window that we're looking at
    ranges = graph_time_series_agg.ranges
    xr = ranges['x']
    x1 = datetime.datetime.fromtimestamp(int(xr[0]/1000.))
    x2 = datetime.datetime.fromtimestamp(int(xr[1]/1000.))

    # Text value that we're looking at
    search_idx = df['Complaint Type'].str.contains(search_complaints_text_input.value).fillna(False)
    windowed_idx = search_idx & (df.index>x1) & (df.index<x2)

    if not on_zoom:
        # Replot the time series aggregate on input change
        grouped = df[search_idx].resample('{}H'.format(slider_val), how='count')

        graph_time_series_agg.restyle({
            'x': [grouped['Complaint Type'].index],
            'y': [grouped['Complaint Type']],
            'type': 'bar'
        })

    
    # Replot the value counts aggregate
    vc = df[windowed_idx][column_name].value_counts()
    
    graph_complaints.restyle({'x': [vc.index], 'y': [vc.values], 'type': 'bar'})

def on_trait_change():
    replot()

search_complaints_text_input.on_trait_change(on_trait_change, 'value')
column_headers_dropdown.on_trait_change(on_trait_change, 'value')
slider.on_trait_change(on_trait_change, 'value')

-c:35: DeprecationWarning:

on_trait_change is deprecated: use observe instead

-c:36: DeprecationWarning:

on_trait_change is deprecated: use observe instead

-c:37: DeprecationWarning:

on_trait_change is deprecated: use observe instead



In [10]:
def replot_complaints_on_zoom(_, ranges):
    graph_time_series_agg.ranges = ranges

    replot(True)

graph_time_series_agg.on_zoom(replot_complaints_on_zoom)


In [11]:
display(slider)
display(search_complaints_text_input)
display(column_headers_dropdown)

display(graph_time_series_agg)
display(graph_complaints)

In [12]:
replot()

In [ ]:


graph_time_series_agg.relayout({'title': 'Number of complaints over time'})
graph_complaints.relayout({'yaxis.title': 'Number of events'})

